In [1]:
# Some libraries are imported.
import pandas as pd
import numpy as np
import re


In [2]:
#Reading the files.
file = pd.read_json('..\data\intervals_challenge.json', lines = True)
df = pd.DataFrame(file)

In [3]:
#The dataframe is shown.
df

,duration,distance,started_at,trip_id,vehicle_id,city_id,type
0,857,5384,2016-10-03 13:00:00.286999941,c00cee6963e0dc66e50e271239426914,52d38cf1a3240d5cbdcf730f2d9a47d6,pentos,driving_to_destination
1,245,1248,2016-10-03 13:00:00.852999926,427425e1f4318ca2461168bdd6e4fcbd,8336b28f24c3e7a1e3d582073b164895,volantis,going_to_pickup
2,1249,5847,2016-10-03 13:00:01.670000076,757867f6d7c00ef92a65bfaa3895943f,8885c59374cc539163e83f01ed59fd16,pentos,driving_to_destination
3,471,2585,2016-10-03 13:00:01.841000080,d09d1301d361f7359d0d936557d10f89,81b63920454f70b6755a494e3b28b3a7,bravos,going_to_pickup
4,182,743,2016-10-03 13:00:01.970000029,00f20a701f0ec2519353ef3ffaf75068,b73030977cbad61c9db55418909864fa,pentos,going_to_pickup
...,...,...,...,...,...,...,...
165165,NA,NA,2016-10-04 20:36:10.846999884,d2f81b419daddb90bd701ab9870f47a3,05e4532a59b7bfe65894335d1bc83e4d,volantis,driving_to_destination
165166,NA,NA,2016-10-04 20:36:12.719000101,73a07d4504036997306e0e3330529294,bed6dd187f23b3e8d956bb0d1ab0524a,volantis,waiting_for_rider
165167,NA,NA,2016-10-04 20:36:14.763000011,d98c047d565bd488b17e08594965c479,31cd2f19d27c0c0c0ca386de72866a81,bravos,driving_to_destination
165168,NA,NA,2016-10-04 20:36:15.957999945,ce9a7ae5f76667c9f92b149a1b345a2d,4a555013634506246eed5d801ffbe526,volantis,driving_to_destination


In [4]:
df.isna().sum()

duration      0
distance      0
started_at    0
trip_id       0
vehicle_id    0
city_id       0
type          0
dtype: int64

In [5]:
df.columns

Index(['duration', 'distance', 'started_at', 'trip_id', 'vehicle_id',
       'city_id', 'type'],
      dtype='object')

In [6]:
df.dtypes

duration              object
distance              object
started_at    datetime64[ns]
trip_id               object
vehicle_id            object
city_id               object
type                  object
dtype: object

In [7]:
df['duration'] = pd.to_numeric(df.duration, errors = 'coerce')
df['distance'] = pd.to_numeric(df.distance, errors = 'coerce')

In [8]:
df.dtypes

duration             float64
distance             float64
started_at    datetime64[ns]
trip_id               object
vehicle_id            object
city_id               object
type                  object
dtype: object

In [9]:
df.isna().sum()

duration      1157
distance      1157
started_at       0
trip_id          0
vehicle_id       0
city_id          0
type             0
dtype: int64

In [10]:
df = df.dropna()

In [11]:
#Lets create a new column
df['distance_type'] = np.where(df.trip_id.str.startswith(tuple(str(i) for i in range(9))), 'road', 'linear')

<ipython-input-11-691c2b02f80f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_type'] = np.where(df.trip_id.str.startswith(tuple(str(i) for i in range(9))), 'road', 'linear')


In [12]:
df.distance_type.value_counts()

road      93004
linear    71009
Name: distance_type, dtype: int64

In [13]:
df.groupby(['distance_type']).describe()

duration                                                   \
                 count        mean         std  min   25%    50%    75%   
distance_type                                                             
linear         71009.0  432.743427  547.788101  0.0  55.0  246.0  612.0   
road           93004.0  433.124532  567.584285  0.0  54.0  245.0  620.0   

                       distance                                              \
                   max    count         mean          std  min   25%    50%   
distance_type                                                                 
linear         19134.0  71009.0  1927.770085  6557.560494  0.0  19.0  607.0   
road           48654.0  93004.0  1956.681670  7705.527123  0.0  19.0  609.0   

                                  
                  75%        max  
distance_type                     
linear         2327.0   892112.0  
road           2365.0  1218089.0

In [26]:
print('Duration:', (1-432.74/433.12)*100,'%')
print('Distance:',(1-1927.77/1956.68)*100,'%')

Duration: 0.08773550055412027 %
Distance: 1.4775027086697867 %


At this first approach, it is difficult to decide, since there is not that much of a big difference. However, I see a difference in "Distance" (even when in very low). 

I'll analyze each city separately.

In [15]:
df.city_id.unique()

array(['pentos', 'volantis', 'bravos'], dtype=object)

In [41]:
def relations(df):
    x = df.groupby(['distance_type']).duration.mean().min()
    y = df.groupby(['distance_type']).duration.mean().max()

    i = df.groupby(['distance_type']).distance.mean().min()
    j = df.groupby(['distance_type']).distance.mean().max()
    
    a = print('Duration:', (1-x/y)*100,'%')
    b = print('Distance:',(1-i/j)*100,'%')
    
    return

In [45]:
df_pentos = df.loc[df.city_id == 'pentos']
df_pentos.groupby(['distance_type']).describe()

duration                                                   \
                 count        mean         std  min   25%    50%    75%   
distance_type                                                             
linear         49121.0  388.195232  487.042042  0.0  47.0  222.0  549.0   
road           63696.0  387.698191  522.147403  0.0  46.0  221.0  555.0   

                       distance                                              \
                   max    count         mean          std  min   25%    50%   
distance_type                                                                 
linear         19134.0  49121.0  1663.108650  3247.896754  0.0  18.0  552.0   
road           48654.0  63696.0  1666.255699  3523.958232  0.0  19.0  560.0   

                                 
                  75%       max  
distance_type                    
linear         2081.0   97334.0  
road           2107.0  374951.0

In [43]:
relations(df_pentos)

Duration: 0.1280388658480236 %
Distance: 0.18886950416927561 %


In [17]:
df_volantis = df.loc[df.city_id == 'volantis']
df_volantis.groupby(['distance_type']).describe() 

duration                                                    \
                 count        mean         std  min   25%    50%     75%   
distance_type                                                              
linear         15068.0  507.000465  673.063314  0.0  80.0  280.0  688.25   
road           20323.0  503.039463  646.025791  0.0  77.0  271.0  694.00   

                       distance                                              \
                   max    count         mean          std  min   25%    50%   
distance_type                                                                 
linear         17903.0  15068.0  2208.099814  4481.021592  0.0  20.0  620.0   
road           14819.0  20323.0  2258.791566  4538.706041  0.0  19.0  599.0   

                                 
                  75%       max  
distance_type                    
linear         2644.0  102700.0  
road           2691.0   87818.0

In [46]:
relations(df_volantis)

Duration: 0.7812619829326017 %
Distance: 2.244197861743824 %


In [18]:
df_bravos = df.loc[df.city_id == 'bravos']
df_bravos.groupby(['distance_type']).describe() 

duration                                                   \
                 count        mean         std  min   25%    50%    75%   
distance_type                                                             
linear          6820.0  589.539150  603.303012  0.0  79.0  462.0  890.0   
road            8985.0  597.019366  636.693272  0.0  73.0  444.0  893.0   

                      distance                                                \
                  max    count         mean           std  min   25%     50%   
distance_type                                                                  
linear         7215.0   6820.0  3214.636070  18030.466360  0.0  23.0  1373.0   
road           9842.0   8985.0  3332.217919  21847.612032  0.0  21.0  1384.0   

                                   
                   75%        max  
distance_type                      
linear         3407.25   892112.0  
road           3473.00  1218089.0

In [47]:
relations(df_bravos)

Duration: 1.2529268697334395 %
Distance: 3.528636218853043 %


## Conclusion

Something we see is that the lower quantity of data, the higher the difference between Linear/Road values.
To get to a further conclusion I would need more data. Also the price of the service, both mine and the GPS query, would be require to get the point where we are achieving the quantity.

In [51]:
df.started_at[2016]



Timestamp('2016-10-03 13:18:39.634999990')